In [1]:
"""
Libraries

"""

import csv
from datetime import datetime
from typing import Callable

import logging

import numpy as np
import csv

import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from scipy.optimize import linear_sum_assignment
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.model_selection import KFold

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
"""
Setting generic hyperparameters

"""

num_epochs: int = 50
batch_size: int = 256 # Should be set to a power of 2.
# Learning rate
lr:         float = 1e-4 # Learning rate used in the IIC paper: lr=1e-4.

"""
GPU utilization

"""
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Specifications
if torch.cuda.is_available():
    print(f"Number of available devices: {torch.cuda.device_count()}\n",
          f"Device name: {torch.cuda.get_device_name(torch.cuda.current_device())}\n",
          f"Total GPU memory device 0: {torch.cuda.get_device_properties(0).total_memory/(1024**3):.2f} GB\n")

Number of available devices: 1
 Device name: NVIDIA A100 80GB PCIe
 Total GPU memory device 0: 79.20 GB



In [4]:
'''
Store data to .csv file

'''

# open the file for writing
f = open(f'logs/{datetime.now().strftime("%Y-%m-%d-%H-%M")}.csv', 'w')
# create a CSV writer object
writer = csv.writer(f)
# write the header row to the CSV file
writer.writerow(['epoch', 'loss', 'running_acc', 'acc', 'running_nmi', 'nmi'])

44

In [5]:
"""
Unsupervised Machine Learning Framework

"""

def train(model, data_loader: DataLoader, criterion: Callable, optimizer: torch.optim, num_epochs: int, num_classes: int=None) -> None:
    """
    Trains a given model using the provided training data, optimizer and loss criterion for a given number of epochs.

    Args:
        model: Neural network model to train.
        data_loader: PyTorch data loader containing the training data.
        criterion: Loss criterion used for training the model.
        optimizer: Optimizer used to update the model's parameters.
        num_epochs: Number of epochs to train the model.

    Returns:
        None
    """

    for epoch in range(num_epochs):

        running_loss = 0.0
        running_acc  = 0.0
        running_nmi  = 0.0

        # Initialize tensors for storing true and predicted labels
        labels_true = torch.zeros(len(data_loader.dataset))
        labels_pred = torch.zeros(len(data_loader.dataset))

        # Loop over the mini-batches in the data loader
        for i, data in enumerate(data_loader):
        
            # Get the inputs and labels for the mini-batch
            inputs, labels = data

            # Use GPU if available
            inputs = inputs.to(device)

            # Image augmentation
            if data_loader.dataset.augment_data:
                inputs_trans = torch.stack([data_loader.dataset.transform_list(input) for input in inputs])
                # # Flatten input data for the feed forward model
                # inputs       = [inputs.view(inputs.size(0), -1), inputs_trans.view(inputs_trans.size(0), -1)]
                inputs       = [inputs, inputs_trans]
            # else:
                # inputs = inputs.view(inputs.size(0), -1)
        
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass through the model
            if data_loader.dataset.augment_data:
                outputs = [F.softmax(model(inputs[0]), dim=1), F.softmax(model(inputs[1]), dim=1)]
            else:
                outputs = [F.softmax(model(inputs), dim=1), 0]

            # Set arguments for objective function
            # kwargs = {key: value for key, value in locals().items() if key in criterion.__code__.co_varnames}
            kwargs = {"model": model, "inputs": inputs, "outputs": outputs}
            kwargs = {key: value for key, value in kwargs.items() if key in criterion.__code__.co_varnames}
            
            # Compute the loss
            loss = criterion(**kwargs)
            # Backward pass through the model and compute gradients
            loss.backward()
        
            # Update the weights
            optimizer.step()

            # Accumulate the loss for the mini-batch
            running_loss += loss.item()

            outputs = outputs[0] #if data_loader.dataset.augment_data else outputs

            running_acc  += unsupervised_clustering_accuracy(labels, torch.argmax(outputs.cpu(), dim=1),C=num_classes)
            running_nmi  += unsupervised_normalized_mutual_info(labels, torch.argmax(outputs.cpu(), dim=1),C=num_classes)

            # Store predicted and true labels in tensors
            labels_true[i*len(labels):(i+1)*len(labels)] = labels
            labels_pred[i*len(labels):(i+1)*len(labels)] = torch.argmax(outputs, dim=1)

        acc = unsupervised_clustering_accuracy(labels_true, labels_pred, C=num_classes)
        nmi = unsupervised_normalized_mutual_info(labels_true, labels_pred, C=num_classes)

        # Compute the average loss and accuracy for the epoch and print
        print(f"Epoch {epoch+1} loss: {running_loss/len(data_loader):.4f},\
              running_acc: {running_acc/len(data_loader):.4f}, acc: {acc:.4f},\
              running_nmi: {running_nmi/len(data_loader):.4f}, nmi: {nmi:.4f}")
        # Store data to file
        writer.writerow([epoch+1, running_loss/len(data_loader), running_acc/len(data_loader), acc, running_nmi/len(data_loader), nmi])

def reassign(y_true: torch.Tensor, y_pred: torch.Tensor, C: int=None) -> float:
    
    # Create confusion matrix
    cm = confusion_matrix(y_pred, y_true, labels=list(range(C)))

    # Compute best matching between true and predicted labels using the Hungarian algorithm
    _, col_ind = linear_sum_assignment(-cm)

    # Reassign labels for the predicted clusters
    y_pred_reassigned = torch.tensor(col_ind)[y_pred.long()]
    
    return y_pred_reassigned
        
def unsupervised_clustering_accuracy(y_true: torch.Tensor, y_pred: torch.Tensor, C: int=None) -> float:
    """
    Computes the unsupervised clustering accuracy between two clusterings.
    Uses the Hungarian algorithm to find the best matching between true and predicted labels.

    Args:
        y_true: true cluster labels as a 1D torch.Tensor
        y_pred: predicted cluster labels as a 1D torch.Tensor
        C:      number of classes

    Returns:
        accuracy: unsupervised clustering accuracy as a float
    """
    
    y_pred_reassigned = reassign(y_true, y_pred, C)

    # Compute accuracy as the percentage of correctly classified samples
    acc = accuracy_score(y_true, y_pred_reassigned)

    return acc

def unsupervised_balanced_clustering_accuracy(y_true: torch.Tensor, y_pred: torch.Tensor, C: int=None) -> float:
    """
    Computes the unsupervised clustering accuracy between two clusterings.
    Uses the Hungarian algorithm to find the best matching between true and predicted labels.

    Args:
        y_true: true cluster labels as a 1D torch.Tensor
        y_pred: predicted cluster labels as a 1D torch.Tensor
        C:      number of classes

    Returns:
        accuracy: unsupervised clustering accuracy as a float
    """
    
    y_pred_reassigned = reassign(y_true, y_pred, C)

    # Compute accuracy as the percentage of correctly classified samples
    acc = balanced_accuracy_score(y_true, y_pred_reassigned)

    return acc

def unsupervised_normalized_mutual_info(y_true: torch.Tensor, y_pred: torch.Tensor, C: int=None) -> float:
    """
    Computes the unsupervised clustering accuracy between two clusterings.
    Uses the Hungarian algorithm to find the best matching between true and predicted labels.

    Args:
        y_true: true cluster labels as a 1D torch.Tensor
        y_pred: predicted cluster labels as a 1D torch.Tensor
        C:      number of classes

    Returns:
        accuracy: unsupervised clustering accuracy as a float
    """
    
    y_pred_reassigned = reassign(y_true, y_pred, C)

    # Compute accuracy as the percentage of correctly classified samples
    acc = normalized_mutual_info_score(y_true, y_pred_reassigned)

    return acc

def test_classifier(model, data_loader: DataLoader, num_classes: int) -> float:
    """
    Testing a classifier given the model and a test set.

    Args:
        model: Neural network model to train.
        data_loader: PyTorch data loader containing the test data.
    
    Returns:
        None
    """
    
    # Disable gradient computation, not needed for inference
    model.eval()
    # Initialize tensors for storing true and predicted labels
    y_true = torch.zeros(len(data_loader.dataset))
    y_pred = torch.zeros(len(data_loader.dataset))

    with torch.no_grad():
        # Iterate over the mini-batches in the data loader
        for i, data in enumerate(data_loader):
            # Get the inputs and true labels for the mini-batch and reshape
            inputs, labels_true = data
            
            # Use GPU if available
            inputs      = inputs.to(device)
                                    
            # # TODO flattening should be done in the feed forward model, else statement should be removed
            # inputs = inputs.view(inputs.size(0), -1)
            
            # Forward pass through the model to get predicted labels
            labels_pred = F.softmax(model(inputs), dim=1)

            # Store predicted and true labels in tensors
            y_pred[i*len(labels_true):(i+1)*len(labels_true)] = torch.argmax(labels_pred.cpu(), dim=1)
            y_true[i*len(labels_true):(i+1)*len(labels_true)] = labels_true

    # Compute unsupervised clustering accuracy score
    acc = unsupervised_clustering_accuracy(y_true, y_pred, C=num_classes)
    
    acc_balanced = unsupervised_balanced_clustering_accuracy(y_true, y_pred, C=num_classes)

    nmi = unsupervised_normalized_mutual_info(y_true, y_pred, C=num_classes)
    
    print(f'acc: {acc}')
    print(f'acc_balanced: {acc_balanced}')
    print(f'nmi: {nmi}')

#    y_reassign = reassign(y_true, y_pred, C=num_classes)
    
    # Let's assume y_true and y_pred are your tensors
#    y_true = y_true.tolist()  # Convert tensors to lists
#    y_pred = y_reassign.tolist()

    # Zip the two lists together
#    data = list(zip(y_true, y_pred))

    # Write the data to a CSV file
#    with open('output.csv', 'w', newline='') as f:
#        writer = csv.writer(f)
#        writer.writerow(["y_true", "y_pred"])  # Write the header
#        writer.writerows(data)  # Write the data rows
    
    return acc, acc_balanced, nmi

In [6]:
"""

"""

from archt import get_model

# Information Maximizing Self-Augmented Training
from IMSAT import regularized_information_maximization

# Invariant Information Clustering
from IIC import invariant_information_clustering

from datasets.dataset_classes import NDSBDataset, MNISTDataset, init_dataset

In [7]:
# Initialize loss function, and optimizer
criterion = invariant_information_clustering

# Store metadata to .log file
logger = logging.getLogger(__name__)
# Set the logging level
logger.setLevel(logging.INFO)
# Add handler to the logger
logger.addHandler(logging.FileHandler(f'logs/{datetime.now().strftime("%Y-%m-%d-%H-%M")}.log'))

# Initialize KFold
kf = KFold(n_splits=5, shuffle=True)

ndsb_img_paths, ndsb_labels = init_dataset("./datasets/NDSB/train", subset=True)

# Convert your lists into numpy arrays
ndsb_img_paths = np.array(ndsb_img_paths)
ndsb_labels = np.array(ndsb_labels)

# Initialize list to store accuracies
accuracies = []
accuracies_balanced = []
normalized_mutual_information = []

# For each split train a new model
for train_index, test_index in kf.split(ndsb_img_paths):
    
    # Get train and test splits
    train_paths, test_paths = ndsb_img_paths[train_index], ndsb_img_paths[test_index]
    train_labels, test_labels = ndsb_labels[train_index], ndsb_labels[test_index]

    # Create the train and test datasets
    train_dataset = NDSBDataset(data_paths=train_paths, data_labels=train_labels, augment_data=True)
    test_dataset  = NDSBDataset(data_paths=test_paths, data_labels=test_labels)

    # Create the train and test data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader  = DataLoader(test_dataset, batch_size=batch_size)

    # Initialize model, loss function, and optimizer
    model = get_model("resnet18", num_classes=10).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Train the model
    train(model, train_loader, criterion, optimizer, num_epochs, num_classes=10)

    # Test model
    acc, acc_balanced, nmi = test_classifier(model, test_loader,num_classes=10)

    # Append accuracy to list
    accuracies.append(acc)
    accuracies_balanced.append(acc_balanced)
    normalized_mutual_information.append(nmi)

# Calculate mean accuracy and standard deviation
mean_acc, std_acc = np.mean(accuracies), np.std(accuracies)

mean_acc_balanced, std_acc_balanced = np.mean(accuracies_balanced), np.std(accuracies_balanced)

mean_nmi, std_nmi = np.mean(normalized_mutual_information), np.std(normalized_mutual_information)

# Write metadata to .log file
logger.info(f'Optimization criterion: {criterion.__name__}')
logger.info(f'Learning rate: {lr}')
logger.info(f'Number of epochs: {num_epochs}')
logger.info(f'Batch size: {batch_size}')
logger.info(f'Optimizer: {optimizer}')
logger.info(f'Model: {model}')

logger.info(f'Accuracy: {mean_acc}')
logger.info(f'    std: {std_acc}')
logger.info(f'Balanced Accuracy: {mean_acc_balanced}')
logger.info(f'    std: {std_acc_balanced}')
logger.info(f'Normalized Mutual Information: {mean_nmi}')
logger.info(f'    std: {std_nmi}')
# Close data file
f.close()

print(f'Mean Accuracy: {mean_acc}, Standard Deviation: {std_acc}')
print(f'Mean Balanced Accuracy: {mean_acc_balanced}, Standard Deviation: {std_acc_balanced}')


Model specifications: ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu):

Epoch 32 loss: -1.0517,              running_acc: 0.4478, acc: 0.4617,              running_nmi: 0.4840, nmi: 0.4732
Epoch 33 loss: -1.0692,              running_acc: 0.4561, acc: 0.4718,              running_nmi: 0.5026, nmi: 0.4851
Epoch 34 loss: -1.0701,              running_acc: 0.4660, acc: 0.4681,              running_nmi: 0.4890, nmi: 0.4734
Epoch 35 loss: -1.0593,              running_acc: 0.4467, acc: 0.4488,              running_nmi: 0.4908, nmi: 0.4674
Epoch 36 loss: -1.0695,              running_acc: 0.4666, acc: 0.4635,              running_nmi: 0.4973, nmi: 0.4727
Epoch 37 loss: -1.0662,              running_acc: 0.4947, acc: 0.5060,              running_nmi: 0.5152, nmi: 0.4900
Epoch 38 loss: -1.0599,              running_acc: 0.4902, acc: 0.4922,              running_nmi: 0.5211, nmi: 0.4953
Epoch 39 loss: -1.0783,              running_acc: 0.4561, acc: 0.4700,              running_nmi: 0.4994, nmi: 0.4895
Epoch 40 loss: -1.0767,              running_acc: 0.4467, acc: 0

Epoch 12 loss: -0.8243,              running_acc: 0.4366, acc: 0.4432,              running_nmi: 0.4840, nmi: 0.4829
Epoch 13 loss: -0.8827,              running_acc: 0.4384, acc: 0.4469,              running_nmi: 0.5246, nmi: 0.5105
Epoch 14 loss: -0.9010,              running_acc: 0.4251, acc: 0.4294,              running_nmi: 0.5086, nmi: 0.4994
Epoch 15 loss: -0.9076,              running_acc: 0.4288, acc: 0.4312,              running_nmi: 0.5044, nmi: 0.4914
Epoch 16 loss: -0.8936,              running_acc: 0.4035, acc: 0.4054,              running_nmi: 0.4702, nmi: 0.4688
Epoch 17 loss: -0.9094,              running_acc: 0.4246, acc: 0.4072,              running_nmi: 0.4603, nmi: 0.4443
Epoch 18 loss: -0.9574,              running_acc: 0.4113, acc: 0.4211,              running_nmi: 0.4805, nmi: 0.4673
Epoch 19 loss: -0.9442,              running_acc: 0.4462, acc: 0.4247,              running_nmi: 0.5026, nmi: 0.4904
Epoch 20 loss: -0.9652,              running_acc: 0.4418, acc: 0

Epoch 1 loss: -0.1060,              running_acc: 0.3009, acc: 0.3047,              running_nmi: 0.2717, nmi: 0.2154
Epoch 2 loss: -0.3418,              running_acc: 0.3251, acc: 0.3315,              running_nmi: 0.3285, nmi: 0.3105
Epoch 3 loss: -0.4831,              running_acc: 0.3222, acc: 0.3333,              running_nmi: 0.3520, nmi: 0.3496
Epoch 4 loss: -0.5563,              running_acc: 0.3762, acc: 0.3601,              running_nmi: 0.3689, nmi: 0.3702
Epoch 5 loss: -0.6382,              running_acc: 0.3746, acc: 0.3924,              running_nmi: 0.4277, nmi: 0.4210
Epoch 6 loss: -0.7115,              running_acc: 0.4210, acc: 0.4109,              running_nmi: 0.4662, nmi: 0.4491
Epoch 7 loss: -0.7303,              running_acc: 0.4254, acc: 0.4284,              running_nmi: 0.4744, nmi: 0.4692
Epoch 8 loss: -0.7842,              running_acc: 0.4475, acc: 0.4266,              running_nmi: 0.4899, nmi: 0.4673
Epoch 9 loss: -0.7857,              running_acc: 0.4178, acc: 0.4303,   

Epoch 1 loss: -0.1186,              running_acc: 0.3303, acc: 0.3149,              running_nmi: 0.3068, nmi: 0.2349
Epoch 2 loss: -0.2792,              running_acc: 0.3824, acc: 0.3934,              running_nmi: 0.3794, nmi: 0.3573
Epoch 3 loss: -0.3536,              running_acc: 0.3555, acc: 0.3887,              running_nmi: 0.3770, nmi: 0.3931
Epoch 4 loss: -0.4281,              running_acc: 0.3548, acc: 0.3592,              running_nmi: 0.3746, nmi: 0.3788
Epoch 5 loss: -0.5168,              running_acc: 0.3587, acc: 0.3564,              running_nmi: 0.4105, nmi: 0.3720
Epoch 6 loss: -0.6161,              running_acc: 0.3681, acc: 0.3527,              running_nmi: 0.4128, nmi: 0.3649
Epoch 7 loss: -0.6889,              running_acc: 0.4025, acc: 0.3518,              running_nmi: 0.4670, nmi: 0.4026
Epoch 8 loss: -0.7407,              running_acc: 0.4165, acc: 0.3647,              running_nmi: 0.4719, nmi: 0.4088
Epoch 9 loss: -0.7766,              running_acc: 0.4056, acc: 0.3638,   

Epoch 1 loss: -0.1208,              running_acc: 0.2805, acc: 0.2989,              running_nmi: 0.2467, nmi: 0.1836
Epoch 2 loss: -0.2910,              running_acc: 0.3106, acc: 0.3238,              running_nmi: 0.2935, nmi: 0.2814
Epoch 3 loss: -0.3552,              running_acc: 0.3616, acc: 0.3561,              running_nmi: 0.3793, nmi: 0.3388
Epoch 4 loss: -0.4697,              running_acc: 0.3517, acc: 0.3358,              running_nmi: 0.3910, nmi: 0.3504
Epoch 5 loss: -0.5325,              running_acc: 0.3552, acc: 0.3533,              running_nmi: 0.3846, nmi: 0.3979
Epoch 6 loss: -0.5909,              running_acc: 0.3645, acc: 0.3699,              running_nmi: 0.4026, nmi: 0.4122
Epoch 7 loss: -0.6486,              running_acc: 0.3829, acc: 0.3921,              running_nmi: 0.4280, nmi: 0.4320
Epoch 8 loss: -0.7060,              running_acc: 0.3936, acc: 0.4105,              running_nmi: 0.4571, nmi: 0.4572
Epoch 9 loss: -0.7642,              running_acc: 0.4142, acc: 0.4225,   

In [8]:
#logger.info(f'lambda: {2}')
#logger.info(f'vet:true')